# MRI Anomaly Detection and Data Interpretation System

This notebook implements an AI-powered system for MRI anomaly detection using LangChain, OpenAI GPT-4o-mini, and Tavily AI API with interactive image upload capabilities.

## Features
- 🖼️ Interactive MRI image upload with drag-and-drop support
- 🧠 AI-powered anomaly detection simulation
- 📚 Medical context search using Tavily AI
- 💬 Detailed explanations using OpenAI GPT-4o-mini via LangChain
- 📄 Professional HTML report generation
- 🔄 Batch processing capabilities
- 📊 Comprehensive error handling and logging

## Important Medical Disclaimer
⚠️ **This system is for demonstration and educational purposes only. It should not be used for actual medical diagnosis. Always consult qualified healthcare professionals for medical advice.**

## 1. Setup and Installation

First, let's install all the required packages for our MRI analysis system.

In [ ]:
# Install required packages
!pip install langchain langchain-openai langchain-community openai requests pillow numpy matplotlib python-dotenv ipywidgets

## 2. Import Required Libraries

Import all necessary libraries including LangChain components, OpenAI, image processing libraries, and Jupyter widgets.

In [ ]:
import os
import requests
import json
from typing import List, Dict, Optional
import base64
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import logging
import io

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Jupyter widgets for file upload
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## 3. Environment Configuration

Set up API keys and environment variables securely.

In [ ]:
# Set up API keys
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"
TAVILY_API_KEY = "your-tavily-api-key-here"

print("✅ API keys configured successfully!")
print("🔐 Environment setup complete!")

## 4. MRI Image Handler Class

Handle MRI image validation, preprocessing, and display functionality.

In [ ]:
class MRIImageHandler:
    """Handles MRI image upload, validation, and preprocessing"""
    
    SUPPORTED_FORMATS = ['jpg', 'jpeg', 'png', 'dicom', 'dcm']
    MAX_SIZE_MB = 10
    
    @staticmethod
    def validate_image(image_path: str = None, image_data: bytes = None) -> bool:
        """Validate image format and size"""
        if image_path:
            if not os.path.exists(image_path):
                raise ValueError(f"Image path does not exist: {image_path}")
            
            # Check file format
            file_ext = image_path.split('.')[-1].lower()
            if file_ext not in MRIImageHandler.SUPPORTED_FORMATS:
                raise ValueError(f"Invalid format. Supported: {', '.join(MRIImageHandler.SUPPORTED_FORMATS)}")
            
            # Check file size
            file_size_mb = os.path.getsize(image_path) / (1024 * 1024)
            if file_size_mb > MRIImageHandler.MAX_SIZE_MB:
                raise ValueError(f"File too large. Maximum size: {MRIImageHandler.MAX_SIZE_MB}MB")
        
        elif image_data:
            # Check data size
            data_size_mb = len(image_data) / (1024 * 1024)
            if data_size_mb > MRIImageHandler.MAX_SIZE_MB:
                raise ValueError(f"File too large. Maximum size: {MRIImageHandler.MAX_SIZE_MB}MB")
        
        return True
    
    @staticmethod
    def encode_image_base64(image_path: str = None, image_data: bytes = None) -> str:
        """Encode image to base64 for API transmission"""
        if image_path:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        elif image_data:
            return base64.b64encode(image_data).decode('utf-8')
        else:
            raise ValueError("Either image_path or image_data must be provided")
    
    @staticmethod
    def display_image(image_path: str = None, image_data: bytes = None, title: str = "MRI Scan"):
        """Display the MRI image"""
        if image_path:
            img = Image.open(image_path)
        elif image_data:
            img = Image.open(io.BytesIO(image_data))
        else:
            raise ValueError("Either image_path or image_data must be provided")
            
        plt.figure(figsize=(10, 10))
        plt.imshow(img, cmap='gray' if img.mode == 'L' else None)
        plt.title(title, fontsize=16)
        plt.axis('off')
        plt.show()
    
    @staticmethod
    def save_uploaded_image(image_data: bytes, filename: str) -> str:
        """Save uploaded image data to file"""
        with open(filename, 'wb') as f:
            f.write(image_data)
        return filename

print("✅ MRI Image Handler class created!")

## 5. Tavily AI Integration Class

Create integration with Tavily AI API for medical context search and MRI analysis simulation.

In [ ]:
class TavilyMRIAnalyzer:
    """Handles MRI analysis using Tavily AI API"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.tavily.com/v1"
        
    def analyze_mri(self, image_path: str = None, image_data: bytes = None) -> Dict:
        """
        Analyze MRI image for anomalies
        Note: Since Tavily is a search API, we'll simulate MRI analysis
        """
        # Validate image
        if image_path:
            MRIImageHandler.validate_image(image_path=image_path)
        elif image_data:
            MRIImageHandler.validate_image(image_data=image_data)
        
        # Simulated analysis results based on random patterns
        import random
        
        # Generate realistic simulated abnormalities
        possible_abnormalities = [
            {
                "type": "lesion",
                "location": "left frontal lobe",
                "size": "12mm x 8mm",
                "confidence": round(random.uniform(0.75, 0.95), 2),
                "characteristics": "hyperintense on T2-weighted images"
            },
            {
                "type": "white matter changes",
                "location": "periventricular region",
                "severity": "mild",
                "confidence": round(random.uniform(0.80, 0.98), 2),
                "characteristics": "scattered punctate foci"
            },
            {
                "type": "microhemorrhage",
                "location": "right parietal lobe",
                "size": "3mm",
                "confidence": round(random.uniform(0.70, 0.90), 2),
                "characteristics": "hypointense on T2* sequences"
            },
            {
                "type": "cortical atrophy",
                "location": "bilateral temporal lobes",
                "severity": "moderate",
                "confidence": round(random.uniform(0.85, 0.95), 2),
                "characteristics": "widened sulci and reduced cortical thickness"
            }
        ]
        
        # Randomly select 1-3 abnormalities for simulation
        num_abnormalities = random.randint(1, 3)
        selected_abnormalities = random.sample(possible_abnormalities, num_abnormalities)
        
        simulated_results = {
            "status": "success",
            "image_id": f"mri_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            "abnormalities": selected_abnormalities,
            "overall_assessment": "Abnormalities detected requiring clinical correlation" if selected_abnormalities else "No significant abnormalities detected"
        }
        
        return simulated_results
    
    def search_medical_context(self, abnormality_type: str) -> List[str]:
        """Use Tavily to search for medical context about detected abnormalities"""
        headers = {
            "Content-Type": "application/json"
        }
        
        payload = {
            "api_key": self.api_key,
            "query": f"MRI {abnormality_type} clinical significance treatment options medical imaging",
            "search_depth": "advanced",
            "max_results": 3
        }
        
        try:
            response = requests.post(
                f"{self.base_url}/search",
                json=payload,
                headers=headers
            )
            
            if response.status_code == 200:
                results = response.json()
                return [result.get('content', '') for result in results.get('results', [])]
            else:
                print(f"Tavily search error: {response.status_code}")
                return []
        except Exception as e:
            print(f"Error searching medical context: {e}")
            return []

print("✅ Tavily MRI Analyzer class created!")

## 6. OpenAI GPT-4 Integration with LangChain

Implement LangChain chains using OpenAI GPT-4o-mini for generating medical explanations.

In [ ]:
class MedicalExplanationGenerator:
    """Generates medical explanations using OpenAI GPT-4o-mini via LangChain"""
    
    def __init__(self):
        self.llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.3,  # Lower temperature for more consistent medical explanations
            max_tokens=1000
        )
        
    def create_explanation_chain(self):
        """Create a LangChain chain for generating medical explanations"""
        
        prompt_template = ChatPromptTemplate.from_template("""
You are an experienced radiologist providing clear, professional explanations of MRI findings.

MRI Analysis Results:
{abnormalities}

Additional Medical Context:
{medical_context}

Please provide:
1. A clear explanation of each detected abnormality
2. The clinical significance of these findings
3. Potential risks or concerns
4. Recommended follow-up actions
5. Important notes for the patient

Format your response with clear sections and use both medical terminology (with explanations) 
and patient-friendly language. Be thorough but accessible.

Remember: This is an AI-generated analysis and should be reviewed by a qualified medical professional.
        """)
        
        # Use the new RunnableSequence syntax instead of deprecated LLMChain
        chain = prompt_template | self.llm
        
        return chain
    
    def generate_explanation(self, abnormalities: List[Dict], medical_context: List[str]) -> str:
        """Generate comprehensive explanation of MRI findings"""
        
        # Format abnormalities for the prompt
        abnormalities_text = self._format_abnormalities(abnormalities)
        context_text = "\n".join(medical_context) if medical_context else "No additional context available"
        
        # Run the chain using the new invoke method
        chain = self.create_explanation_chain()
        result = chain.invoke({
            "abnormalities": abnormalities_text,
            "medical_context": context_text
        })
        
        # Extract content from the result
        explanation = result.content if hasattr(result, 'content') else str(result)
        
        return explanation
    
    def _format_abnormalities(self, abnormalities: List[Dict]) -> str:
        """Format abnormalities data for the prompt"""
        if not abnormalities:
            return "No abnormalities detected in this MRI scan."
            
        formatted = []
        for i, abnormality in enumerate(abnormalities, 1):
            formatted.append(f"""
Abnormality {i}:
- Type: {abnormality.get('type', 'Unknown')}
- Location: {abnormality.get('location', 'Not specified')}
- Size/Severity: {abnormality.get('size', abnormality.get('severity', 'Not specified'))}
- Confidence: {abnormality.get('confidence', 0):.0%}
- Characteristics: {abnormality.get('characteristics', 'Not specified')}
            """.strip())
        
        return "\n\n".join(formatted)

print("✅ Medical Explanation Generator class created!")

## 7. Report Generation System

Generate professional HTML reports with comprehensive medical documentation.

In [ ]:
class MRIReportGenerator:
    """Generates comprehensive MRI analysis reports"""
    
    @staticmethod
    def generate_html_report(
        image_filename: str,
        analysis_results: Dict,
        explanation: str,
        patient_id: str = "Anonymous"
    ) -> str:
        """Generate an HTML report with findings"""
        
        report_date = datetime.now().strftime("%B %d, %Y at %I:%M %p")
        
        html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <title>MRI Analysis Report</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
            line-height: 1.6;
            background-color: #f8f9fa;
        }}
        .container {{
            background-color: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 10px;
            margin-bottom: 25px;
            text-align: center;
        }}
        .header h1 {{
            margin: 0;
            font-size: 2.2em;
        }}
        .section {{
            margin-bottom: 30px;
            padding: 20px;
            background-color: #f8f9fa;
            border-radius: 8px;
            border-left: 5px solid #007bff;
        }}
        .abnormality {{
            background-color: #fff3cd;
            padding: 18px;
            margin: 15px 0;
            border-radius: 8px;
            border-left: 5px solid #ffc107;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }}
        .explanation {{
            background-color: #e8f4f8;
            padding: 25px;
            border-radius: 8px;
            margin-top: 20px;
            border-left: 5px solid #17a2b8;
        }}
        .disclaimer {{
            background-color: #f8d7da;
            padding: 20px;
            border-radius: 8px;
            border-left: 5px solid #dc3545;
            color: #721c24;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 15px;
            background-color: white;
            border-radius: 8px;
            overflow: hidden;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }}
        th, td {{
            text-align: left;
            padding: 12px 15px;
            border-bottom: 1px solid #e9ecef;
        }}
        th {{
            background-color: #495057;
            color: white;
            font-weight: 600;
        }}
        .confidence-high {{ color: #28a745; font-weight: bold; }}
        .confidence-medium {{ color: #ffc107; font-weight: bold; }}
        .confidence-low {{ color: #dc3545; font-weight: bold; }}
        .summary-stats {{
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
        }}
        .stat-card {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            min-width: 120px;
        }}
        .stat-number {{
            font-size: 2em;
            font-weight: bold;
            color: #007bff;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🧠 MRI Analysis Report</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.1em;">AI-Powered Medical Image Analysis</p>
        </div>
        
        <div class="section">
            <h2>📋 Report Information</h2>
            <table>
                <tr><td><strong>Patient ID:</strong></td><td>{patient_id}</td></tr>
                <tr><td><strong>Report Date:</strong></td><td>{report_date}</td></tr>
                <tr><td><strong>Image File:</strong></td><td>{image_filename}</td></tr>
                <tr><td><strong>Analysis ID:</strong></td><td>{analysis_results.get('image_id', 'N/A')}</td></tr>
            </table>
        </div>
        
        <div class="section">
            <h2>📊 Analysis Summary</h2>
            <div class="summary-stats">
                <div class="stat-card">
                    <div class="stat-number">{len(analysis_results.get('abnormalities', []))}</div>
                    <div>Abnormalities</div>
                </div>
                <div class="stat-card">
                    <div class="stat-number">{analysis_results.get('status', 'Unknown').title()}</div>
                    <div>Status</div>
                </div>
            </div>
            <p><strong>Overall Assessment:</strong> {analysis_results.get('overall_assessment', 'Analysis complete')}</p>
        </div>
        
        <div class="section">
            <h2>🔍 Detected Abnormalities</h2>
            {MRIReportGenerator._format_abnormalities_html(analysis_results.get('abnormalities', []))}
        </div>
        
        <div class="section">
            <h2>💡 Detailed Medical Interpretation</h2>
            <div class="explanation">
                {explanation.replace(chr(10), '<br>')}
            </div>
        </div>
        
        <div class="section disclaimer">
            <h2>⚠️ Important Medical Disclaimer</h2>
            <p style="font-weight: bold; margin-bottom: 15px;">
                This report is generated by an AI system for demonstration and educational purposes only.
            </p>
            <ul>
                <li>This analysis should <strong>NEVER</strong> be used for actual medical diagnosis</li>
                <li>Always consult qualified healthcare professionals for medical advice</li>
                <li>This system is not a substitute for professional medical evaluation</li>
                <li>Any medical decisions should be made only by licensed physicians</li>
                <li>The AI may produce inaccurate or incomplete results</li>
            </ul>
        </div>
        
        <div style="text-align: center; margin-top: 30px; color: #6c757d;">
            <p><small>Generated by MRI Anomaly Detection System | AI-Powered Medical Analysis Tool</small></p>
        </div>
    </div>
</body>
</html>
        """
        
        return html_template
    
    @staticmethod
    def _format_abnormalities_html(abnormalities: List[Dict]) -> str:
        """Format abnormalities as HTML"""
        if not abnormalities:
            return "<div class='abnormality'><h3>✅ No Abnormalities Detected</h3><p>The AI analysis did not detect any significant abnormalities in this MRI scan.</p></div>"
        
        html_parts = []
        for i, abnormality in enumerate(abnormalities, 1):
            confidence = abnormality.get('confidence', 0)
            confidence_class = 'confidence-high' if confidence >= 0.8 else 'confidence-medium' if confidence >= 0.6 else 'confidence-low'
            
            html_parts.append(f"""
            <div class="abnormality">
                <h3>🔸 Abnormality {i}: {abnormality.get('type', 'Unknown').title()}</h3>
                <table>
                    <tr><td><strong>📍 Location:</strong></td><td>{abnormality.get('location', 'Not specified')}</td></tr>
                    <tr><td><strong>📏 Size/Severity:</strong></td><td>{abnormality.get('size', abnormality.get('severity', 'Not specified'))}</td></tr>
                    <tr><td><strong>🎯 Confidence:</strong></td><td><span class="{confidence_class}">{abnormality.get('confidence', 0):.0%}</span></td></tr>
                    <tr><td><strong>🔬 Characteristics:</strong></td><td>{abnormality.get('characteristics', 'Not specified')}</td></tr>
                </table>
            </div>
            """)
        
        return "".join(html_parts)
    
    @staticmethod
    def save_report(html_content: str, output_path: str = "mri_report.html"):
        """Save the HTML report to a file"""
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        return output_path

print("✅ MRI Report Generator class created!")

## 8. Main System Integration Class

Create the main orchestrating class that combines all components into a complete MRI analysis pipeline.

In [ ]:
class MRIAnalysisSystem:
    """Main system integrating all components"""
    
    def __init__(self, tavily_api_key: str):
        self.image_handler = MRIImageHandler()
        self.tavily_analyzer = TavilyMRIAnalyzer(tavily_api_key)
        self.explanation_generator = MedicalExplanationGenerator()
        self.report_generator = MRIReportGenerator()
        
        # Setup logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('mri_analysis.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger('MRIAnalysisSystem')
    
    def analyze_mri_image(self, image_path: str = None, image_data: bytes = None, 
                         patient_id: str = "Anonymous", filename: str = "uploaded_image.jpg") -> Dict:
        """
        Complete MRI analysis pipeline
        """
        print("🔍 Starting MRI Analysis Pipeline...")
        self.logger.info(f"Starting analysis for patient: {patient_id}")
        
        try:
            # Step 1: Validate and display image
            print("\n📸 Validating image...")
            if image_path:
                self.image_handler.validate_image(image_path=image_path)
                self.image_handler.display_image(image_path=image_path, title="📋 Original MRI Scan")
                image_filename = os.path.basename(image_path)
            elif image_data:
                self.image_handler.validate_image(image_data=image_data)
                self.image_handler.display_image(image_data=image_data, title="📋 Uploaded MRI Scan")
                image_filename = filename
            else:
                raise ValueError("Either image_path or image_data must be provided")
            
            # Step 2: Analyze MRI with Tavily (simulated)
            print("\n🧠 Analyzing MRI for anomalies...")
            analysis_results = self.tavily_analyzer.analyze_mri(image_path=image_path, image_data=image_data)
            num_abnormalities = len(analysis_results.get('abnormalities', []))
            print(f"✓ Analysis complete: Found {num_abnormalities} potential abnormalit{'ies' if num_abnormalities != 1 else 'y'}")
            
            # Step 3: Get medical context for each abnormality type
            print("\n📚 Gathering medical context from research...")
            medical_contexts = []
            for abnormality in analysis_results.get('abnormalities', []):
                print(f"  🔍 Researching: {abnormality['type']}")
                context = self.tavily_analyzer.search_medical_context(abnormality['type'])
                medical_contexts.extend(context)
            
            # Step 4: Generate detailed explanation
            print("\n💬 Generating medical explanation with AI...")
            explanation = self.explanation_generator.generate_explanation(
                analysis_results.get('abnormalities', []),
                medical_contexts
            )
            
            # Step 5: Generate report
            print("\n📄 Creating comprehensive report...")
            html_report = self.report_generator.generate_html_report(
                image_filename,
                analysis_results,
                explanation,
                patient_id
            )
            
            # Save report
            report_filename = f"mri_report_{patient_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
            report_path = self.report_generator.save_report(html_report, report_filename)
            
            # Log success
            self.logger.info(f"Analysis completed successfully. Report: {report_path}")
            
            # Return results
            return {
                "success": True,
                "analysis_results": analysis_results,
                "explanation": explanation,
                "report_path": report_path,
                "html_report": html_report
            }
            
        except Exception as e:
            error_msg = f"Error during analysis: {str(e)}"
            print(f"\n❌ {error_msg}")
            self.logger.error(error_msg, exc_info=True)
            return {
                "success": False,
                "error": str(e)
            }
    
    def display_results_summary(self, results: Dict):
        """Display a summary of the analysis results"""
        if results['success']:
            print("\n" + "="*60)
            print("✅ MRI ANALYSIS COMPLETE")
            print("="*60)
            
            analysis = results['analysis_results']
            abnormalities = analysis.get('abnormalities', [])
            
            print(f"\n📊 Summary:")
            print(f"   • Abnormalities found: {len(abnormalities)}")
            print(f"   • Report saved to: {results['report_path']}")
            print(f"   • Analysis ID: {analysis.get('image_id', 'N/A')}")
            
            if abnormalities:
                print(f"\n🔍 Detected Abnormalities:")
                for i, abnormality in enumerate(abnormalities, 1):
                    conf = abnormality.get('confidence', 0)
                    print(f"   {i}. {abnormality.get('type', 'Unknown').title()}")
                    print(f"      Location: {abnormality.get('location', 'Not specified')}")
                    print(f"      Confidence: {conf:.0%}")
            
            print(f"\n📋 Overall Assessment: {analysis.get('overall_assessment', 'Complete')}")
            print(f"\n⚠️  Please have this report reviewed by a qualified medical professional.")
            print(f"    This AI analysis is for demonstration purposes only.")
        else:
            print(f"\n❌ Analysis failed: {results['error']}")

print("✅ Main MRI Analysis System class created!")

## 9. Image Upload Interface

Create interactive file upload widgets with drag-and-drop functionality for MRI scans.

In [ ]:
class MRIUploadInterface:
    """Interactive interface for MRI upload and analysis"""
    
    def __init__(self, analysis_system: MRIAnalysisSystem):
        self.analysis_system = analysis_system
        self.uploaded_files = {}
        self.results = None
        
        # Create widgets
        self.file_upload = widgets.FileUpload(
            accept='.jpg,.jpeg,.png,.dicom,.dcm',
            multiple=False,
            description='Upload MRI'
        )
        self.patient_id_input = widgets.Text(
            value='Anonymous',
            placeholder='Enter Patient ID',
            description='Patient ID:',
            disabled=False
        )
        self.analyze_button = widgets.Button(
            description='Analyze MRI',
            disabled=True,
            button_style='primary',
            tooltip='Click to analyze the uploaded MRI',
            icon='search'
        )
        self.view_report_button = widgets.Button(
            description='View Full Report',
            disabled=True,
            button_style='info',
            tooltip='Click to view the full HTML report',
            icon='file-text'
        )
        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progress:',
            bar_style='info',
            orientation='horizontal'
        )
        self.status_label = widgets.HTML(value="<p>Please upload an MRI scan to begin.</p>")
        self.output_area = widgets.Output()
        
        # Register callbacks
        self.file_upload.observe(self._on_file_upload, names='value')
        self.analyze_button.on_click(self._on_analyze_click)
        self.view_report_button.on_click(self._on_view_report_click)

    def _on_file_upload(self, change):
        """Handle file upload event"""
        self.status_label.value = "<p style='color: #007bff;'>Processing upload...</p>"
        with self.output_area:
            clear_output()
            
        try:
            # The 'new' value from the FileUpload widget
            uploaded_value = change['new']
            
            if not uploaded_value:
                self.uploaded_files = {}
                self.analyze_button.disabled = True
                self.status_label.value = "<p>Upload cancelled. Please upload an MRI scan.</p>"
                return

            # Handle new tuple-based format from ipywidgets
            if isinstance(uploaded_value, tuple):
                if not uploaded_value:
                     raise ValueError("File upload value is an empty tuple.")
                file_info = uploaded_value[0] # Get the first file info object
                filename = file_info.name
                file_content = file_info.content
            # Handle old dict-based format for backward compatibility
            elif isinstance(uploaded_value, dict):
                filename = list(uploaded_value.keys())[0]
                file_content = uploaded_value[filename]['content']
            else:
                raise TypeError(f"Unexpected type for uploaded file value: {type(uploaded_value)}")

            # Store file data
            self.uploaded_files = {filename: file_content}
            
            # Validate image
            MRIImageHandler.validate_image(image_data=file_content)
            
            # Update UI
            self.analyze_button.disabled = False
            self.status_label.value = f"<p style='color: #28a745;'>✅ Ready to analyze: <strong>{filename}</strong></p>"
            
            # Display thumbnail
            with self.output_area:
                print("🖼️ Uploaded Image Preview:")
                MRIImageHandler.display_image(image_data=file_content, title=filename)

        except Exception as e:
            error_msg = f"❌ Upload error: {str(e)}"
            self.status_label.value = f"<p style='color: #dc3545;'>{error_msg}</p>"
            with self.output_area:
                print(error_msg)
                import traceback
                traceback.print_exc()
            self.analyze_button.disabled = True

    def _on_analyze_click(self, button):
        """Handle analyze button click"""
        # Check if files are uploaded
        if not self.uploaded_files:
            self.status_label.value = "<p style='color: #dc3545;'>❌ Please upload a file first</p>"
            return
        
        # Disable button during analysis
        self.analyze_button.disabled = True
        self.view_report_button.disabled = True
        
        # Clear previous output
        with self.output_area:
            clear_output()
        
        # Start progress
        self.progress_bar.value = 10
        self.status_label.value = "<p style='color: #007bff;'>🔄 Starting analysis...</p>"
        
        try:
            # Get the uploaded file
            filename = list(self.uploaded_files.keys())[0]
            file_data = self.uploaded_files[filename]
            
            # Validate file data
            if not file_data or len(file_data) == 0:
                raise ValueError("File appears to be empty")
            
            self.progress_bar.value = 30
            self.status_label.value = "<p style='color: #007bff;'>🧠 AI analyzing image...</p>"
            
            # Run analysis
            patient_id = self.patient_id_input.value or 'Anonymous'
            results = self.analysis_system.analyze_mri_image(
                image_data=file_data,
                patient_id=patient_id,
                filename=filename
            )
            
            self.progress_bar.value = 90
            
            # Store results
            self.results = results
            
            # Display results in output area
            with self.output_area:
                if results['success']:
                    self.analysis_system.display_results_summary(results)
                else:
                    print(f"❌ Analysis failed: {results['error']}")
            
            self.progress_bar.value = 100
            
            if results['success']:
                self.status_label.value = "<p style='color: #28a745;'>✅ Analysis complete! Report generated.</p>"
                self.view_report_button.disabled = False
            else:
                self.status_label.value = f"<p style='color: #dc3545;'>❌ Analysis failed: {results.get('error', 'Unknown error')}</p>"
                
        except Exception as e:
            with self.output_area:
                print(f"❌ Error during analysis: {str(e)}")
                import traceback
                traceback.print_exc()
            self.status_label.value = f"<p style='color: #dc3545;'>❌ Error: {str(e)}</p>"
            self.progress_bar.value = 0
        
        finally:
            # Re-enable button
            self.analyze_button.disabled = False

    def _on_view_report_click(self, button):
        """Handle view report button click"""
        with self.output_area:
            clear_output()
            if self.results and self.results['success']:
                print("📄 Displaying Full HTML Report...")
                display(HTML(self.results['html_report']))
            else:
                print("❌ No report available to display.")
    
    def create_upload_widget(self):
        """Create and return the full widget layout"""
        
        # Layout
        controls = widgets.VBox([
            self.patient_id_input,
            self.file_upload,
            self.status_label,
            self.progress_bar,
            widgets.HBox([self.analyze_button, self.view_report_button])
        ])
        
        return widgets.VBox([
            widgets.HTML("<h2>🖼️ MRI Anomaly Detection Interface</h2>"),
            controls,
            self.output_area
        ])

print("✅ MRI Upload Interface class created!")

## 10. Analysis Pipeline Execution

Initialize the system and create the interactive interface for MRI analysis.

In [ ]:
# Initialize the MRI Analysis System
print("🚀 Initializing MRI Analysis System...")

# Create the main analysis system
mri_system = MRIAnalysisSystem(tavily_api_key=TAVILY_API_KEY)

# Create the upload interface
upload_interface = MRIUploadInterface(mri_system)

# Create and display the interface
interface_widget = upload_interface.create_upload_widget()

print("✅ System initialized successfully!")
print("📱 Interactive interface ready below...")

In [ ]:
# Display the interactive MRI upload and analysis interface
display(interface_widget)

In [ ]:
## 10.1. AI Chat Interface

print("Interactive chat interface for asking medical questions using Tavily AI research and OpenAI GPT-4o-mini responses.")

In [ ]:
class AIChatInterface:
    """Interactive chat interface for medical questions using Tavily AI and OpenAI GPT-4o-mini"""
    
    def __init__(self, tavily_api_key: str):
        self.tavily_api_key = tavily_api_key
        self.llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.7,  # Slightly higher for more conversational responses
            max_tokens=1500
        )
        self.chat_history = []
        
        # Create widgets
        self.question_input = widgets.Textarea(
            value='',
            placeholder='Ask any medical or MRI-related question...',
            description='Question:',
            disabled=False,
            rows=3,
            layout=widgets.Layout(width='100%')
        )
        
        self.ask_button = widgets.Button(
            description='Ask AI',
            disabled=False,
            button_style='success',
            tooltip='Get AI-powered answer with research',
            icon='question-circle'
        )
        
        self.research_toggle = widgets.Checkbox(
            value=True,
            description='Include Tavily Research',
            disabled=False,
            tooltip='Include web research in the response'
        )
        
        self.clear_button = widgets.Button(
            description='Clear Chat',
            disabled=False,
            button_style='warning',
            tooltip='Clear chat history',
            icon='trash'
        )
        
        self.chat_output = widgets.Output()
        
        # Register callbacks
        self.ask_button.on_click(self._on_ask_click)
        self.clear_button.on_click(self._on_clear_click)
        self.question_input.observe(self._on_enter_key, names='value')
    
    def _on_enter_key(self, change):
        """Handle Enter key in question input (Ctrl+Enter to submit)"""
        # Note: In Jupyter, we can't easily detect Ctrl+Enter, but users can click the button
        pass
    
    def _search_tavily(self, question: str) -> str:
        """Search Tavily AI for relevant information"""
        headers = {
            "Content-Type": "application/json"
        }
        
        payload = {
            "api_key": self.tavily_api_key,
            "query": f"medical health {question}",
            "search_depth": "advanced",
            "max_results": 5,
            "include_answer": True
        }
        
        try:
            response = requests.post(
                "https://api.tavily.com/v1/search",
                json=payload,
                headers=headers
            )
            
            if response.status_code == 200:
                results = response.json()
                
                # Combine answer and top results
                research_content = ""
                
                # Include the direct answer if available
                if results.get('answer'):
                    research_content += f"Direct Answer: {results['answer']}\n\n"
                
                # Include top search results
                if results.get('results'):
                    research_content += "Additional Research:\n"
                    for i, result in enumerate(results['results'][:3], 1):
                        research_content += f"{i}. {result.get('title', 'No title')}\n"
                        research_content += f"   {result.get('content', 'No content')[:200]}...\n\n"
                
                return research_content
            else:
                return f"Research unavailable (Status: {response.status_code})"
                
        except Exception as e:
            return f"Research error: {str(e)}"
    
    def _generate_response(self, question: str, research_data: str = "") -> str:
        """Generate AI response using OpenAI GPT-4o-mini"""
        
        # Create context-aware prompt
        if research_data:
            prompt_text = f"""You are a knowledgeable medical AI assistant. A user has asked the following question, and I've gathered some research information to help inform your response.

User Question: {question}

Research Information:
{research_data}

Please provide a comprehensive, accurate, and helpful response based on the question and research information. Follow these guidelines:

1. Be informative but accessible to general audiences
2. Include relevant medical terminology with explanations
3. Mention when something requires professional medical consultation
4. Be balanced and evidence-based
5. Include appropriate medical disclaimers when discussing health conditions
6. Format your response clearly with sections if needed

Important: Always include a disclaimer that this is for educational purposes only and not a substitute for professional medical advice.

Response:"""
        else:
            prompt_text = f"""You are a knowledgeable medical AI assistant. Please answer the following question:

{question}

Guidelines:
1. Provide accurate, evidence-based information
2. Be accessible to general audiences while including relevant medical terms
3. Always mention when professional medical consultation is needed
4. Include appropriate disclaimers about educational vs. medical advice
5. Be comprehensive but concise

Important: Always include a disclaimer that this is for educational purposes only and not a substitute for professional medical advice.

Response:"""
        
        try:
            # Use the LLM to generate response
            prompt_template = ChatPromptTemplate.from_template(prompt_text)
            chain = prompt_template | self.llm
            result = chain.invoke({})
            
            response = result.content if hasattr(result, 'content') else str(result)
            return response
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
    
    def _on_ask_click(self, button):
        """Handle ask button click"""
        question = self.question_input.value.strip()
        
        if not question:
            with self.chat_output:
                print("❌ Please enter a question first.")
            return
        
        # Disable button during processing
        self.ask_button.disabled = True
        self.ask_button.description = 'Processing...'
        
        with self.chat_output:
            print(f"\n{'='*60}")
            print(f"🙋 Question: {question}")
            print(f"{'='*60}")
            
            # Get research if enabled
            research_data = ""
            if self.research_toggle.value:
                print("🔍 Searching for relevant research...")
                research_data = self._search_tavily(question)
                print("✅ Research completed.")
            
            # Generate AI response
            print("🤖 Generating AI response...")
            response = self._generate_response(question, research_data)
            
            print("\n💬 AI Response:")
            print("-" * 40)
            print(response)
            
            # Add to chat history
            self.chat_history.append({
                'question': question,
                'research': research_data if self.research_toggle.value else None,
                'response': response,
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
            
            print(f"\n📝 Added to chat history (Total: {len(self.chat_history)} conversations)")
        
        # Clear input and re-enable button
        self.question_input.value = ''
        self.ask_button.disabled = False
        self.ask_button.description = 'Ask AI'
    
    def _on_clear_click(self, button):
        """Handle clear chat button click"""
        self.chat_history = []
        with self.chat_output:
            clear_output()
            print("🧹 Chat history cleared!")
    
    def create_chat_widget(self):
        """Create and return the chat interface widget"""
        
        # Controls layout
        controls = widgets.VBox([
            self.question_input,
            widgets.HBox([
                self.research_toggle,
                self.ask_button,
                self.clear_button
            ]),
        ])
        
        return widgets.VBox([
            widgets.HTML("<h3>🤖 AI Medical Assistant</h3>"),
            widgets.HTML("<p><em>Ask questions about medical topics, MRI imaging, health conditions, and more!</em></p>"),
            controls,
            self.chat_output
        ])
    
    def export_chat_history(self) -> str:
        """Export chat history as formatted text"""
        if not self.chat_history:
            return "No chat history to export."
        
        export_text = "AI Medical Chat History\n"
        export_text += "=" * 50 + "\n\n"
        
        for i, chat in enumerate(self.chat_history, 1):
            export_text += f"Conversation {i} - {chat['timestamp']}\n"
            export_text += "-" * 30 + "\n"
            export_text += f"Question: {chat['question']}\n\n"
            
            if chat['research']:
                export_text += f"Research Data:\n{chat['research']}\n\n"
            
            export_text += f"AI Response:\n{chat['response']}\n\n"
            export_text += "=" * 50 + "\n\n"
        
        return export_text

print("✅ AI Chat Interface class created!")

In [ ]:
# Initialize and display the AI Chat Interface
print("🤖 Initializing AI Chat Interface...")

# Create the chat interface
chat_interface = AIChatInterface(tavily_api_key=TAVILY_API_KEY)

# Create and display the chat widget
chat_widget = chat_interface.create_chat_widget()

print("✅ AI Chat Interface ready!")
print("💬 Ask any medical or MRI-related questions below:")

# Display the chat interface
display(chat_widget)

In [ ]:
# Utility functions for the AI Chat Interface

def save_chat_history(filename: str = None):
    """Save chat history to a file"""
    if filename is None:
        filename = f"chat_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    
    history_text = chat_interface.export_chat_history()
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(history_text)
    
    print(f"💾 Chat history saved to: {filename}")
    return filename

def get_chat_stats():
    """Get statistics about the chat sessions"""
    if not chat_interface.chat_history:
        print("📊 No chat history available.")
        return
    
    total_conversations = len(chat_interface.chat_history)
    conversations_with_research = sum(1 for chat in chat_interface.chat_history if chat['research'])
    
    print("📊 Chat Statistics:")
    print("=" * 30)
    print(f"Total Conversations: {total_conversations}")
    print(f"With Research: {conversations_with_research}")
    print(f"Without Research: {total_conversations - conversations_with_research}")
    
    if chat_interface.chat_history:
        first_chat = chat_interface.chat_history[0]['timestamp']
        last_chat = chat_interface.chat_history[-1]['timestamp']
        print(f"First Question: {first_chat}")
        print(f"Latest Question: {last_chat}")

def ask_quick_question(question: str, include_research: bool = True):
    """Quick function to ask a question programmatically"""
    print(f"🤖 Quick Question: {question}")
    print("=" * 50)
    
    # Get research if requested
    research_data = ""
    if include_research:
        print("🔍 Searching for research...")
        research_data = chat_interface._search_tavily(question)
    
    # Generate response
    print("💭 Generating response...")
    response = chat_interface._generate_response(question, research_data)
    
    print("\n💬 Response:")
    print("-" * 30)
    print(response)
    
    # Add to history
    chat_interface.chat_history.append({
        'question': question,
        'research': research_data if include_research else None,
        'response': response,
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })

# Example questions you can try:
example_questions = [
    "What are the common types of brain lesions visible on MRI?",
    "How does contrast enhancement work in MRI imaging?",
    "What is the difference between T1 and T2 weighted MRI images?",
    "What are the early signs of multiple sclerosis on MRI?",
    "How accurate is AI in detecting brain tumors on MRI scans?",
    "What should patients know before getting an MRI scan?",
    "What are the limitations of MRI in diagnosing neurological conditions?"
]

print("✅ Chat utility functions created!")
print(f"\n💡 Example questions to try:")
for i, q in enumerate(example_questions, 1):
    print(f"   {i}. {q}")

print(f"\n🔧 Available functions:")
print("   • save_chat_history() - Save conversation history to file")
print("   • get_chat_stats() - View chat statistics")
print("   • ask_quick_question('your question') - Ask questions programmatically")

## 11. Batch Processing Implementation

Process multiple MRI images simultaneously with progress tracking.

In [ ]:
# Example usage function
def run_batch_example():
    """Example of how to run batch processing"""
    # Example with sample file paths (update these with your actual image paths)
    sample_images = [
        "sample_mri_1.jpg",
        "sample_mri_2.jpg", 
        "sample_mri_3.jpg"
    ]
    
    # Filter to only existing files
    existing_images = [img for img in sample_images if os.path.exists(img)]
    
    if existing_images:
        print(f"Found {len(existing_images)} existing images for batch processing...")
        batch_results = batch_process_mri_images(existing_images, mri_system)
        return batch_results
    else:
        print("⚠️  No sample images found. Create some test images first or update the file paths.")
        return []

print("✅ Batch processing functions created!")
print("💡 Use run_batch_example() to test batch processing with sample images")

## 12. Error Handling and Logging

Comprehensive error handling and logging systems for robust operation.

In [ ]:
def safe_analyze_mri(image_path: str, system: MRIAnalysisSystem, 
                    patient_id: str = "Anonymous") -> Dict:
    """Analyze MRI with comprehensive error handling"""
    logger = logging.getLogger('SafeMRIAnalysis')
    
    try:
        logger.info(f"Starting safe analysis for: {image_path}")
        
        # Validate inputs
        if not image_path or not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
        
        # Run analysis
        results = system.analyze_mri_image(image_path, patient_id=patient_id)
        
        if results['success']:
            logger.info(f"Analysis completed successfully. Report: {results['report_path']}")
        else:
            logger.error(f"Analysis failed: {results['error']}")
            
        return results
        
    except FileNotFoundError as e:
        error_msg = f"File error: {str(e)}"
        logger.error(error_msg)
        return {"success": False, "error": error_msg, "error_type": "file_not_found"}
        
    except ValueError as e:
        error_msg = f"Validation error: {str(e)}"
        logger.error(error_msg)
        return {"success": False, "error": error_msg, "error_type": "validation_error"}
        
    except Exception as e:
        error_msg = f"Unexpected error: {str(e)}"
        logger.exception(error_msg)
        return {"success": False, "error": error_msg, "error_type": "unexpected_error"}

def view_system_logs(num_lines: int = 20):
    """View recent system logs"""
    log_file = 'mri_analysis.log'
    
    if os.path.exists(log_file):
        print(f"📋 Last {num_lines} log entries:")
        print("="*60)
        
        with open(log_file, 'r') as f:
            lines = f.readlines()
            for line in lines[-num_lines:]:
                print(line.strip())
    else:
        print("📝 No log file found yet. Run an analysis first.")

def system_health_check(system: MRIAnalysisSystem) -> Dict:
    """Perform a system health check"""
    print("🏥 Performing System Health Check...")
    print("="*50)
    
    health_status = {
        "overall_status": "healthy",
        "checks": {},
        "warnings": [],
        "errors": []
    }
    
    # Check API keys
    print("🔐 Checking API keys...")
    if os.environ.get("OPENAI_API_KEY"):
        health_status["checks"]["openai_key"] = "✅ Present"
    else:
        health_status["checks"]["openai_key"] = "❌ Missing"
        health_status["errors"].append("OpenAI API key not configured")
    
    if system.tavily_analyzer.api_key:
        health_status["checks"]["tavily_key"] = "✅ Present"
    else:
        health_status["checks"]["tavily_key"] = "❌ Missing"
        health_status["errors"].append("Tavily API key not configured")
    
    # Check system components
    print("🔧 Checking system components...")
    try:
        system.explanation_generator.llm
        health_status["checks"]["llm_connection"] = "✅ Ready"
    except Exception as e:
        health_status["checks"]["llm_connection"] = "❌ Error"
        health_status["errors"].append(f"LLM connection error: {str(e)}")
    
    # Check file permissions
    print("📁 Checking file permissions...")
    try:
        test_file = "test_write_permission.tmp"
        with open(test_file, 'w') as f:
            f.write("test")
        os.remove(test_file)
        health_status["checks"]["file_permissions"] = "✅ Write access OK"
    except Exception as e:
        health_status["checks"]["file_permissions"] = "❌ No write access"
        health_status["errors"].append(f"File permission error: {str(e)}")
    
    # Determine overall status
    if health_status["errors"]:
        health_status["overall_status"] = "error"
    elif health_status["warnings"]:
        health_status["overall_status"] = "warning"
    
    # Display results
    print(f"\n🎯 Overall Status: {health_status['overall_status'].upper()}")
    print("\n📊 Component Status:")
    for check, status in health_status["checks"].items():
        print(f"   • {check}: {status}")
    
    if health_status["warnings"]:
        print(f"\n⚠️  Warnings ({len(health_status['warnings'])}):")
        for warning in health_status["warnings"]:
            print(f"   • {warning}")
    
    if health_status["errors"]:
        print(f"\n❌ Errors ({len(health_status['errors'])}):")
        for error in health_status["errors"]:
            print(f"   • {error}")
    
    return health_status

print("✅ Error handling and logging functions created!")
print("💡 Use system_health_check(mri_system) to check system status")

## 13. System Testing Functions

Test functions with sample data generation and system validation.

In [ ]:
def create_sample_mri_images(num_images: int = 3) -> List[str]:
    """Create sample MRI-like images for testing"""
    print(f"🖼️  Creating {num_images} sample MRI images for testing...")
    
    image_paths = []
    
    for i in range(num_images):
        # Create a realistic-looking MRI simulation
        # Brain-like circular structure with noise
        size = 512
        image = np.zeros((size, size), dtype=np.uint8)
        
        # Create brain outline
        center = size // 2
        brain_radius = size // 3
        y, x = np.ogrid[:size, :size]
        brain_mask = (x - center)**2 + (y - center)**2 <= brain_radius**2
        
        # Add brain tissue with varying intensities
        brain_tissue = np.random.normal(120, 30, (size, size))
        brain_tissue = np.clip(brain_tissue, 0, 255).astype(np.uint8)
        image[brain_mask] = brain_tissue[brain_mask]
        
        # Add some random "lesions" for variety
        if i > 0:  # Add lesions to some images
            num_lesions = np.random.randint(1, 4)
            for _ in range(num_lesions):
                lesion_x = np.random.randint(center-brain_radius//2, center+brain_radius//2)
                lesion_y = np.random.randint(center-brain_radius//2, center+brain_radius//2)
                lesion_size = np.random.randint(5, 15)
                
                y_lesion, x_lesion = np.ogrid[:size, :size]
                lesion_mask = (x_lesion - lesion_x)**2 + (y_lesion - lesion_y)**2 <= lesion_size**2
                image[lesion_mask] = 200  # Bright lesion
        
        # Add noise
        noise = np.random.normal(0, 10, (size, size))
        image = np.clip(image + noise, 0, 255).astype(np.uint8)
        
        # Save image
        filename = f"sample_mri_{i+1}.jpg"
        Image.fromarray(image, mode='L').save(filename)
        image_paths.append(filename)
        
        print(f"   ✅ Created: {filename}")
    
    print(f"✨ Successfully created {len(image_paths)} sample images!")
    return image_paths

def test_system_components():
    """Test individual system components"""
    print("🧪 Testing System Components...")
    print("="*50)
    
    test_results = {
        "image_handler": False,
        "tavily_analyzer": False,
        "explanation_generator": False,
        "report_generator": False,
        "main_system": False
    }
    
    try:
        # Test Image Handler
        print("1️⃣  Testing Image Handler...")
        handler = MRIImageHandler()
        # Create a tiny test image
        test_img = np.random.randint(0, 255, (100, 100), dtype=np.uint8)
        test_path = "test_component.jpg"
        Image.fromarray(test_img).save(test_path)
        
        handler.validate_image(test_path)
        encoded = handler.encode_image_base64(test_path)
        assert len(encoded) > 0
        os.remove(test_path)
        test_results["image_handler"] = True
        print("   ✅ Image Handler: PASSED")
        
    except Exception as e:
        print(f"   ❌ Image Handler: FAILED - {str(e)}")
    
    try:
        # Test Tavily Analyzer
        print("2️⃣  Testing Tavily Analyzer...")
        analyzer = TavilyMRIAnalyzer(TAVILY_API_KEY)
        # Test with fake image data
        fake_results = analyzer.analyze_mri(image_data=b"fake_data")
        assert "abnormalities" in fake_results
        test_results["tavily_analyzer"] = True
        print("   ✅ Tavily Analyzer: PASSED")
        
    except Exception as e:
        print(f"   ❌ Tavily Analyzer: FAILED - {str(e)}")
    
    try:
        # Test Explanation Generator
        print("3️⃣  Testing Explanation Generator...")
        generator = MedicalExplanationGenerator()
        test_abnormalities = [{
            "type": "test_lesion",
            "location": "test_location", 
            "confidence": 0.85
        }]
        explanation = generator.generate_explanation(test_abnormalities, [])
        assert len(explanation) > 0
        test_results["explanation_generator"] = True
        print("   ✅ Explanation Generator: PASSED")
        
    except Exception as e:
        print(f"   ❌ Explanation Generator: FAILED - {str(e)}")
    
    try:
        # Test Report Generator
        print("4️⃣  Testing Report Generator...")
        generator = MRIReportGenerator()
        test_data = {
            "abnormalities": [{"type": "test", "location": "test"}],
            "overall_assessment": "test"
        }
        report = generator.generate_html_report("test.jpg", test_data, "test explanation")
        assert "<html>" in report
        test_results["report_generator"] = True
        print("   ✅ Report Generator: PASSED")
        
    except Exception as e:
        print(f"   ❌ Report Generator: FAILED - {str(e)}")
    
    try:
        # Test Main System
        print("5️⃣  Testing Main System...")
        system = MRIAnalysisSystem(TAVILY_API_KEY)
        assert system.image_handler is not None
        assert system.tavily_analyzer is not None
        test_results["main_system"] = True
        print("   ✅ Main System: PASSED")
        
    except Exception as e:
        print(f"   ❌ Main System: FAILED - {str(e)}")
    
    # Summary
    passed = sum(test_results.values())
    total = len(test_results)
    
    print(f"\n📊 Test Summary: {passed}/{total} components passed")
    
    if passed == total:
        print("🎉 All tests passed! System is ready for use.")
    else:
        print("⚠️  Some tests failed. Check error messages above.")
    
    return test_results

def run_full_system_test():
    """Run a complete end-to-end system test"""
    print("🚀 Running Full System Test...")
    print("="*60)
    
    try:
        # Step 1: Create sample images
        print("📸 Step 1: Creating sample images...")
        sample_images = create_sample_mri_images(2)
        
        # Step 2: Test system health
        print("\n🏥 Step 2: System health check...")
        health = system_health_check(mri_system)
        
        if health["overall_status"] == "error":
            print("❌ System health check failed. Fix errors before continuing.")
            return False
        
        # Step 3: Test individual components
        print("\n🧪 Step 3: Component testing...")
        component_results = test_system_components()
        
        # Step 4: Test with sample image
        print("\n🔬 Step 4: End-to-end analysis test...")
        if sample_images:
            test_image = sample_images[0]
            results = safe_analyze_mri(test_image, mri_system, "TEST_PATIENT")
            
            if results["success"]:
                print("✅ End-to-end test PASSED!")
                mri_system.display_results_summary(results)
            else:
                print(f"❌ End-to-end test FAILED: {results['error']}")
                return False
        
        # Cleanup
        print("\n🧹 Cleaning up test files...")
        for img in sample_images:
            if os.path.exists(img):
                os.remove(img)
                print(f"   🗑️  Removed: {img}")
        
        print("\n🎉 Full system test completed successfully!")
        return True
        
    except Exception as e:
        print(f"\n❌ Full system test failed: {str(e)}")
        return False

print("✅ Testing functions created!")
print("🧪 Use run_full_system_test() to run comprehensive system tests")

## 14. System Demonstration and Usage

### Quick Start Guide

1. **📱 Interactive Mode**: Use the upload interface above to analyze MRI images
2. **🧪 Test the System**: Run `run_full_system_test()` to verify everything works
3. **🔍 Health Check**: Run `system_health_check(mri_system)` to check system status
4. **📊 Batch Processing**: Use `batch_process_mri_images()` for multiple images

### Usage Examples

```python
# Test the system
run_full_system_test()

# Check system health
system_health_check(mri_system)

# Create sample images for testing
sample_images = create_sample_mri_images(3)

# Analyze a single image safely
results = safe_analyze_mri("path/to/your/mri.jpg", mri_system, "PATIENT_001")

# View system logs
view_system_logs(10)
```

### Important Notes

- 🔐 **API Keys**: Your OpenAI and Tavily API keys are configured in this notebook
- 📸 **Supported Formats**: JPG, JPEG, PNG, DICOM, DCM files up to 10MB
- 🏥 **Medical Disclaimer**: This is for demonstration purposes only - not for actual medical diagnosis
- 📄 **Reports**: HTML reports are automatically generated and saved
- 🔄 **Logging**: All operations are logged to `mri_analysis.log`

### Next Steps for Production Use

1. **Replace Simulated Analysis**: Integrate with real medical imaging APIs
2. **DICOM Support**: Add proper DICOM file parsing
3. **Security**: Implement proper authentication and data encryption
4. **HIPAA Compliance**: Add healthcare data protection measures
5. **Database Integration**: Store results in medical databases
6. **Web Interface**: Create a proper web application using Streamlit/Gradio

In [ ]:
# Test the system
run_full_system_test()

# Check system health
system_health_check(mri_system)

# Create sample images for testing
sample_images = create_sample_mri_images(3)

In [ ]:
# Analyze a single image safely
results = safe_analyze_mri("C:/Users/ahpuh/Desktop/hg/sample_mri_1.jpg", mri_system, "PATIENT_001")

# View system logs
view_system_logs(10)

In [ ]:
# Run a quick system demonstration
print("🎬 MRI Anomaly Detection System - Quick Demo")
print("="*50)

# Perform system health check
print("🏥 Checking system health...")
health_status = system_health_check(mri_system)

if health_status["overall_status"] != "error":
    print("\n🧪 Running component tests...")
    test_results = test_system_components()
    
    print("\n✨ System is ready for use!")
    print("\n📱 Use the interactive interface above to upload and analyze MRI images.")
    print("🔬 Or run run_full_system_test() for a complete demonstration.")
else:
    print("\n⚠️  Please fix system errors before using the MRI analysis system.")